# Resource Estimation for Integer Factoring

In this notebook we calculate resource estimates for a 2048-bit integer
factoring application based on the implementation described in [[Quantum 5, 433
(2021)](https://quantum-journal.org/papers/q-2021-04-15-433/)].  Our
implementation incorporates all techniques described in the paper, except for
carry runways.  As tolerated error budget, we choose $\epsilon = 1/3$.

For this application, we compute the physical resource estimates directly from
precomputed logical resource estimates.  You can use this notebook as a blueprint to estimate the physical resources required of any application for which you already have computed logical estimates.

## Getting started

We import Python classes from `azure.quantum` to create a workspace and the resource estimator target.

In [ ]:
from azure.quantum import Workspace
from azure.quantum.target.microsoft import MicrosoftEstimator, QubitParams, QECScheme
import qsharp

We connect to the Azure Quantum workspace.

In [ ]:
workspace = Workspace (
    resource_id = "",
    location = ""
)

## Extracting resource estimates from logical resource counts

We create a Q# program that describes the algorithm in terms of precomputed logical resource estimates.  To this end, we are using the [AccountForEstimates](https://learn.microsoft.com/qsharp/api/qsharp/microsoft.quantum.resourceestimation.accountforestimates) operation.

In [ ]:
%%qsharp
open Microsoft.Quantum.ResourceEstimation;

operation FactoringFromLogicalCounts() : Unit {
    use qubits = Qubit[12581];
    
    AccountForEstimates(
        [TCount(12), RotationCount(12), RotationDepth(12),
         CczCount(3731607428), MeasurementCount(1078154040)],
        PSSPCLayout(), qubits);
}

## Running experiments

As configurations for the experiment we use all six pre-defined qubit
parameters.  As pre-defined QEC scheme we are using `surface_code` with
gate-based qubit parameters, and `floquet_code` with Majorana based qubit
parameters.  For all experiments we assume an error budget of 1/3. We
leverage the `make_params` function on the `estimator` by setting `num_items` to
6 to create a batching job.  The `error_budget` is set globally for all items,
where as names for `qubit_params` and `qec_scheme` are set individually for each
item.

In [ ]:
estimator = MicrosoftEstimator(workspace)

labels = ["Gate-based µs, 10⁻³", "Gate-based µs, 10⁻⁴", "Gate-based ns, 10⁻³", "Gate-based ns, 10⁻⁴", "Majorana ns, 10⁻⁴", "Majorana ns, 10⁻⁶"]

params = estimator.make_params(num_items=6)
params.error_budget = 0.333
params.items[0].qubit_params.name = QubitParams.GATE_US_E3
params.items[1].qubit_params.name = QubitParams.GATE_US_E4
params.items[2].qubit_params.name = QubitParams.GATE_NS_E3
params.items[3].qubit_params.name = QubitParams.GATE_NS_E4
params.items[4].qubit_params.name = QubitParams.MAJ_NS_E4
params.items[4].qec_scheme.name = QECScheme.FLOQUET_CODE
params.items[5].qubit_params.name = QubitParams.MAJ_NS_E6
params.items[5].qec_scheme.name = QECScheme.FLOQUET_CODE

Next we are submitting the resource estimation job based on the Q# operation above.

In [ ]:
job = estimator.submit(FactoringFromLogicalCounts, input_params=params)
results = job.get_results()


Finally, we are presenting the experimental results using a summary table.

In [ ]:
results.summary_data_frame(labels=labels)

The numbers in the table match the numbers in the paper [Assessing requirements
for scaling quantum computers to real-world impact](https://aka.ms/AQ/RE/Paper).
Feel free to use this table as a starting point for your own experiments.  For
example, you can use other or customized qubit parameters.